## П1. Краевая задача в круге

1. Написать функцию, которая решает краевую задачу в круге радиуса $R$: 

$$
\begin{cases}
- \Delta u + {\alpha}u &= f(x,y),\quad  0 < x^2 + y^2 < R
\\
u\bigg|_{x^2 + y^2 = R,\; x < 0} &= h(x,y), 
\\
\frac{du}{dn}\bigg|_{x^2 + y^2 = R,\; x > 0} &= g(x,y), 
\end{cases}
$$

2. Для решения при помощи пакета `dolfinx` необходимо привести задачу к вариационной постановке следующего вида. Необходимо найти $u \in V$ такую, что 
$$
a(u, v) = L(v), \; \forall v \in V,
$$
где $V = H^1(\Omega)$ - функциональное пространство, а 
$$
\begin{align*}
a(u,v) &:= \int_\Omega \nabla u \nabla v \,dx + \alpha \int_\Omega u v \,dx
\\
L(v) &:= \int_\Omega f v \,dx + \int_{d \Omega} g  v \,ds.
\end{align*}
$$
Здесь 
$$
\Omega = \{(x,y): 0 < x^2 + y^2 < R \} \;\text{(круг радиуса $R$)},
$$
$$
d\Omega = \{(x,y):  x^2 + y^2 = R, \; x > 0 \} \; \text{("правая" граница круга)}
$$

In [1]:
import importlib.util

if importlib.util.find_spec("petsc4py") is not None:
    import dolfinx

from mpi4py import MPI
import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, grad, inner

import gmsh
gmsh.initialize()

import pyvista
from dolfinx.plot import vtk_mesh
pyvista.start_xvfb()

3. Выбор конкретных конечно-элементных пространств $V$  в FEniCS:

3.1 Задание расчетной области (сетки)

In [2]:
# Константы 

R = 1
alpha = 1

In [3]:
# Круг радиуса R с центром в начале координат
circle = gmsh.model.occ.addDisk(0, 0, 0, R, R)

gmsh.model.occ.synchronize()
gdim = 2
gmsh.model.addPhysicalGroup(gdim, [circle], 1)
gmsh.option.setNumber('Mesh.CharacteristicLengthMin', 0.05)
gmsh.option.setNumber('Mesh.CharacteristicLengthMax', 0.05)
gmsh.model.mesh.generate(gdim)

# Сетка
mesh, _, _ = io.gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, 0, gdim=gdim)
# Координаты сетки
x = ufl.SpatialCoordinate(mesh)

Info    : Meshing 1D...
Info    : Meshing curve 1 (Ellipse)
Info    : Done meshing 1D (Wall 0.00014736s, CPU 0.000208s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0568682s, CPU 0.057154s)
Info    : 1550 nodes 3099 elements


3.2 Задание типа функциональных пространств (степень и тип)

In [4]:
V = fem.functionspace(mesh, ('Lagrange', 1))

left_boundary = fem.locate_dofs_geometrical(
    V, lambda x: x[0] < 0 & np.isclose(np.sqrt(x[0]**2 + x[1]**2), R))

4. Решить задачу, визуализировать решение, посчитать погрешность в узлах сетки.

4.1 Выбор тестовых функций

Выберем следующие тестовые функции. В первую очередь выбирается $h(x,y)$. Например, как 

$$
h(x,y) = \sin(x) * \cos(y).
$$

Если $h(x,y)$ является точным решением $u(x,y)$ поставленной задачи, то $g(x,y)$ и $f(x,y)$ вычисляются следующим способом:

$$
\begin{align*}
f & =-\Delta u + {\alpha}u = -\Delta h + \alpha h,\\ 
g & =-{du \over dn} = -{dh \over dn}.
\end{align*}
$$

Итого имеем:
$$
\begin{align*}
h(x,y) &= \sin(x) * \cos(y), \\ 
f(x,y) & = 2 * \cos(y) * \sin(x) + \alpha * \sin(x) * \cos(y), \\
g(x,y) & = - \cos(x) * \cos(y) + \sin(x) * \sin(y).
\end{align*}
$$


In [5]:
# x[0] ~ x, x[1] ~ y

h = ufl.sin(x[0]) * ufl.cos(x[1])
f = 2 * ufl.cos(x[1]) * ufl.sin(x[0]) + alpha * ufl.sin(x[0]) * ufl.cos(x[1])
g = - ufl.cos(x[0]) * ufl.cos(x[1]) + ufl.sin(x[0]) * ufl.sin(x[1])

4.2 Решение задачи

In [6]:
# Задаем граничное условие Дирихле u = h(x,y) на "левой" границе круга
u_lbc = fem.Function(V)
u_lbc.interpolate(fem.Expression(h, V.element.interpolation_points()))
lbc = fem.dirichletbc(u_lbc, left_boundary)

In [14]:
# Задаем дискретную вариационную задачу
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx + alpha * ufl.dot(u, v) * ufl.dx
L = f * v * ufl.dx - g * v * ufl.ds
problem = LinearProblem(a, L, bcs=[lbc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

In [15]:
# Получаем численное решение
uh = problem.solve()

4.3 Визуализация численного решения

In [16]:
pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = uh.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('uh', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x7119c15235e0_2" src="http://95.131.149.198:1489/…

4.4 Получаем точное решение $u(x,y)$ как интерполяцию $h(x,y)$ на сетке

In [17]:
u_sol = fem.Function(V)
u_sol.interpolate(fem.Expression(h, V.element.interpolation_points()))

In [18]:
pyvista.set_jupyter_backend('client')

pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data['u'] = u_sol.x.array
grid.set_active_scalars('u')

plotter = pyvista.Plotter()
plotter.add_text('u', position='upper_edge', font_size=14, color='black')
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

plotter.show()

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x7119c304be80_3" src="http://95.131.149.198:1489/…

4.5 Вычисляем погрешности

In [24]:
# Ошибка по норме L2
error_L2 = fem.assemble_scalar(fem.form((uh - u_sol)**2 * ufl.dx))
error_L2 = np.sqrt(MPI.COMM_WORLD.allreduce(error_L2, op=MPI.SUM))

# Ошибка по максимум-норме
u_vertex_values = uh.x.array
u_sol_vertex_values = u_sol.x.array
error_max = np.max(np.abs(u_vertex_values - u_sol_vertex_values))
error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)

print ('Отклонение численного решения от точного по максимум-норме: ', error_L2)
print ('Отклонение численного решения от точного по L2-норме: ', error_max)

Отклонение численного решения от точного по максимум-норме:  0.2191844309966279
Отклонение численного решения от точного по L2-норме:  0.4953236695433375
